# Import library

In [1]:
import os
import os.path as op
import numpy as np
import matplotlib.pyplot as plt
  
import mne
import nibabel as nib
from mne.datasets import sample
from mne.minimum_norm import make_inverse_operator, apply_inverse_epochs
from mne.datasets import fetch_fsaverage
from scipy.io import loadmat
from scipy.spatial import Delaunay

import gc

%matplotlib qt

In [2]:
img = nib.load("/Users/ivanl/Downloads/MRIcron_windows/MRIcron/Resources/templates/brodmann.nii.gz")

brodmann_data = img.get_fdata()
brodmann_motor = brodmann_data.reshape(-1) == 4
print(brodmann_motor)

shape, affine = img.shape[:3], img.affine
coords = np.array(np.meshgrid(*(range(i) for i in shape), indexing='ij'))
coords = np.rollaxis(coords, 0, len(shape) + 1)
mm_coords = nib.affines.apply_affine(affine, coords)

def in_hull(p, hull):
    """
    Test if points in `p` are in `hull`

    `p` should be a `NxK` coordinates of `N` points in `K` dimensions
    `hull` is either a scipy.spatial.Delaunay object or the `MxK` array of the 
    coordinates of `M` points in `K`dimensions for which Delaunay triangulation
    will be computed
    """
    if not isinstance(hull,Delaunay):
        hull = Delaunay(hull)

    return hull.find_simplex(p)>=0

my_left_points = None
my_right_points = None

[False False False ... False False False]


In [3]:
# cd to google drive
os.chdir("G:")

# Download fsaverage files
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)

# The files live in:
subject = 'fsaverage'
trans = 'fsaverage'  # MNE has a built-in fsaverage transformation
src = op.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif')
bem = op.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')

0 files missing from root.txt in C:\Users\ivanl\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\ivanl\mne_data\MNE-fsaverage-data\fsaverage


In [4]:
source = mne.read_source_spaces(src)
left = source[0]
right = source[1]
left_pos = left["rr"][left["inuse"]==1]
right_pos = right["rr"][right["inuse"]==1]
                        
transformation = mne.read_trans(op.join(fs_dir, "bem", "fsaverage-trans.fif"))

    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read


In [5]:
save_path = op.join(os.getcwd(), "Shared drives", "Motor Imagery", "Source Estimate")

# Load preprocessed data

In [6]:
data_path = "Shared drives/Motor Imagery/resting state eeg & fmri/EEG_MPILMBB_LEMON/EEG_Preprocessed_BIDS_ID/EEG_Preprocessed/"
dirs = os.listdir(data_path)
#files = ["sub-010165_EO.set", "sub-010165_EC.set"]

for file in dirs:
    if file[-3:] == "fdt":
        continue
        
    save_folder = op.join(save_path, file[4:10])
    if not op.exists(save_folder):
        os.makedirs(save_folder)
        
    input_fname = op.join(data_path, file)

    # Load preprocessed data
    raw = mne.io.read_raw_eeglab(input_fname, preload=True, verbose=False)
    
    # Set montage
    # Read and set the EEG electrode locations
    montage = mne.channels.make_standard_montage('standard_1005')
    raw.set_montage(montage)
    
    # Set common average reference
    raw.set_eeg_reference('average', projection=True, verbose=False)
    print(raw.info)
    
    # Construct epochs
    events, _ = mne.events_from_annotations(raw, verbose=False)
    raw.info["events"] = events
    if "EO" in file:
        event_id = {"eyes open": 1}
    elif "EC" in file:
        event_id = {"eyes close": 2}
    tmin, tmax = 0., 2.  # in s
    baseline = None
    epochs = mne.Epochs(
        raw, events=events,
        event_id=event_id, tmin=tmin,
        tmax=tmax, baseline=baseline, verbose=False)
    #epochs.plot()
    print(len(epochs.events))
    noise_cov = mne.compute_covariance(epochs, tmax=0., method=['shrunk', 'empirical'], rank=None, verbose=False)
    del raw # save memory
    
    # Check that the locations of EEG electrodes is correct with respect to MRI
    #mne.viz.plot_alignment(
    #    epochs.info, src=src, eeg=['original', 'projected'], trans=trans,
    #    show_axes=True, mri_fiducials=True, dig='fiducials')
    
    fwd = mne.make_forward_solution(epochs.info, trans=trans, src=src,
                                bem=bem, eeg=True, meg=False, mindist=5.0, n_jobs=1)
    print(fwd)
    
    # Use fwd to compute the sensitivity map for illustration purposes
    #eeg_map = mne.sensitivity_map(fwd, ch_type='eeg', mode='fixed')
    #brain = eeg_map.plot(time_label='EEG sensitivity', subjects_dir=subjects_dir,
    #                     clim=dict(lims=[5, 50, 100]))

    inverse_operator = make_inverse_operator(
        epochs.info, fwd, noise_cov, loose=0.2, depth=0.8)
    del fwd # save memory
    
    method = "dSPM"
    snr = 3.
    lambda2 = 1. / snr ** 2
    stc = apply_inverse_epochs(epochs, inverse_operator, lambda2,
                                  method=method, pick_ori=None, verbose=True)
    del epochs # save memory
    
    # get motor region points (once)
    if my_left_points is None and my_right_points is None:
        my_source = stc[0]
        mni_lh = mne.vertex_to_mni(my_source.vertices[0], 0, subject)
        print(mni_lh.shape)
        mni_rh = mne.vertex_to_mni(my_source.vertices[1], 1, subject)
        print(mni_rh.shape)
        
        """
        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(projection='3d')
        ax.scatter(mm_coords.reshape(-1, 3)[brodmann_motor][:, 0], mm_coords.reshape(-1, 3)[brodmann_motor][:, 1], mm_coords.reshape(-1, 3)[brodmann_motor][:, 2], s=15, marker='|')
        ax.scatter(mni_lh[:, 0], mni_lh[:, 1], mni_lh[:, 2], s=15, marker='_')
        ax.scatter(mni_rh[:, 0], mni_rh[:, 1], mni_rh[:, 2], s=15, marker='_')
        ax.set_xlabel('X Label')
        ax.set_ylabel('Y Label')
        ax.set_zlabel('Z Label')
        plt.show()
        """
        
        my_left_points = in_hull(mni_lh, mm_coords.reshape(-1, 3)[brodmann_motor])
        my_right_points = in_hull(mni_rh, mm_coords.reshape(-1, 3)[brodmann_motor])
        
        mni_left_motor = mne.vertex_to_mni(my_source.vertices[0][my_left_points], 0, subject)
        print(mni_left_motor.shape)
        mni_right_motor = mne.vertex_to_mni(my_source.vertices[1][my_right_points], 1, subject)
        print(mni_right_motor.shape)

        """
        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(projection='3d')
        ax.scatter(mni_lh[:, 0], mni_lh[:, 1], mni_lh[:, 2], s=15, marker='|')
        ax.scatter(mni_rh[:, 0], mni_rh[:, 1], mni_rh[:, 2], s=15, marker='_')
        ax.scatter(mni_left_motor[:, 0], mni_left_motor[:, 1], mni_left_motor[:, 2], s=15, marker='o')
        ax.scatter(mni_right_motor[:, 0], mni_right_motor[:, 1], mni_right_motor[:, 2], s=15, marker='^')
        ax.set_xlabel('X Label')
        ax.set_ylabel('Y Label')
        ax.set_zlabel('Z Label')
        plt.show()
        """
    # slice data
    left_hemi_data = []
    right_hemi_data = []
    for source in stc:
        left_hemi_data.append(source.data[:len(source.vertices[0])][my_left_points])
        right_hemi_data.append(source.data[-len(source.vertices[1]):][my_right_points])
    left_hemi_data = np.array(left_hemi_data)
    right_hemi_data = np.array(right_hemi_data)
    print(left_hemi_data.shape, right_hemi_data.shape)

    if "EO" in file:
        np.savez_compressed(op.join(save_folder, file[4:10]+"_EO.npz"), left=left_hemi_data, right=right_hemi_data)
    elif "EC" in file:
        np.savez_compressed(op.join(save_folder, file[4:10]+"_EC.npz"), left=left_hemi_data, right=right_hemi_data)
    del left_hemi_data, right_hemi_data, stc # save memory
    gc.collect()


C:\Users\ivanl\AppData\Local\Temp/ipykernel_148/1243094924.py:16: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(input_fname, preload=True, verbose=False)
C:\Users\ivanl\AppData\Local\Temp/ipykernel_148/1243094924.py:16: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(input_fname, preload=True, verbose=False)
C:\Users\ivanl\AppData\Local\Temp/ipykernel_148/1243094924.py:42: RuntimeWarning: Epochs are not baseline corrected, covariance matrix may be inaccurate
  noise_cov = mne.compute_covariance(epochs, tmax=0., method=['shrunk', 'empirical'], rank=None, verbose=False)


<Info | 9 non-empty values
 bads: []
 ch_names: Fp1, Fp2, F7, F3, Fz, F4, F8, FC5, FC1, FC2, FC6, T7, C3, Cz, ...
 chs: 61 EEG
 custom_ref_applied: False
 dig: 64 items (3 Cardinal, 61 EEG)
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 61
 projs: Average EEG reference: off
 sfreq: 250.0 Hz
>
225


C:\Users\ivanl\AppData\Local\Temp/ipykernel_148/1243094924.py:42: RuntimeWarning: Too few samples (required : 310 got : 224), covariance estimate may be unreliable
  noise_cov = mne.compute_covariance(epochs, tmax=0., method=['shrunk', 'empirical'], rank=None, verbose=False)


Source space          : C:\Users\ivanl\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-ico-5-src.fif
MRI -> head transform : C:\Users\ivanl\anaconda3\envs\mne\lib\site-packages\mne\data\fsaverage\fsaverage-trans.fif
Measurement data      : instance of Info
Conductor model   : C:\Users\ivanl\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-5120-5120-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Reading C:\Users\ivanl\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-ico-5-src.fif...
Read 2 source spaces a total of 20484 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999994  0.003552  0.000202      -1.76 mm
    -0.003558  0.998389  0.056626      31.09 mm
    -0.000001 -0.056626  0.998395      39.60 mm
     0.000000  0.000000  0.000000       1.00

Read  61 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.

Setting up the BEM 

combining the current components...
Processing epoch : 72 / 224
combining the current components...
Processing epoch : 73 / 224
combining the current components...
Processing epoch : 74 / 224
combining the current components...
Processing epoch : 75 / 224
combining the current components...
Processing epoch : 76 / 224
combining the current components...
Processing epoch : 77 / 224
combining the current components...
Processing epoch : 78 / 224
combining the current components...
Processing epoch : 79 / 224
combining the current components...
Processing epoch : 80 / 224
combining the current components...
Processing epoch : 81 / 224
combining the current components...
Processing epoch : 82 / 224
combining the current components...
Processing epoch : 83 / 224
combining the current components...
Processing epoch : 84 / 224
combining the current components...
Processing epoch : 85 / 224
combining the current components...
Processing epoch : 86 / 224
combining the current components...
Proc

...

<Info | 9 non-empty values
 bads: []
 ch_names: Fp1, Fp2, F7, F3, Fz, F4, F8, FC5, FC1, FC2, FC6, T7, C3, Cz, ...
 chs: 60 EEG
 custom_ref_applied: False
 dig: 63 items (3 Cardinal, 60 EEG)
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 60
 projs: Average EEG reference: off
 sfreq: 250.0 Hz
>
239


C:\Users\ivanl\AppData\Local\Temp/ipykernel_148/1243094924.py:42: RuntimeWarning: Epochs are not baseline corrected, covariance matrix may be inaccurate
  noise_cov = mne.compute_covariance(epochs, tmax=0., method=['shrunk', 'empirical'], rank=None, verbose=False)
C:\Users\ivanl\AppData\Local\Temp/ipykernel_148/1243094924.py:42: RuntimeWarning: Too few samples (required : 305 got : 238), covariance estimate may be unreliable
  noise_cov = mne.compute_covariance(epochs, tmax=0., method=['shrunk', 'empirical'], rank=None, verbose=False)


Source space          : C:\Users\ivanl\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-ico-5-src.fif
MRI -> head transform : C:\Users\ivanl\anaconda3\envs\mne\lib\site-packages\mne\data\fsaverage\fsaverage-trans.fif
Measurement data      : instance of Info
Conductor model   : C:\Users\ivanl\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-5120-5120-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Reading C:\Users\ivanl\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-ico-5-src.fif...
Read 2 source spaces a total of 20484 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999994  0.003552  0.000202      -1.76 mm
    -0.003558  0.998389  0.056626      31.09 mm
    -0.000001 -0.056626  0.998395      39.60 mm
     0.000000  0.000000  0.000000       1.00

Read  60 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.

Setting up the BEM 

combining the current components...
Processing epoch : 72 / 238
combining the current components...
Processing epoch : 73 / 238
combining the current components...
Processing epoch : 74 / 238
combining the current components...
Processing epoch : 75 / 238
combining the current components...
Processing epoch : 76 / 238
combining the current components...
Processing epoch : 77 / 238
combining the current components...
Processing epoch : 78 / 238
combining the current components...
Processing epoch : 79 / 238
combining the current components...
Processing epoch : 80 / 238
combining the current components...
Processing epoch : 81 / 238
combining the current components...
Processing epoch : 82 / 238
combining the current components...
Processing epoch : 83 / 238
combining the current components...
Processing epoch : 84 / 238
combining the current components...
Processing epoch : 85 / 238
combining the current components...
Processing epoch : 86 / 238
combining the current components...
Proc

Processing epoch : 198 / 238
combining the current components...
Processing epoch : 199 / 238
combining the current components...
Processing epoch : 200 / 238
combining the current components...
Processing epoch : 201 / 238
combining the current components...
Processing epoch : 202 / 238
combining the current components...
Processing epoch : 203 / 238
combining the current components...
Processing epoch : 204 / 238
combining the current components...
Processing epoch : 205 / 238
combining the current components...
Processing epoch : 206 / 238
combining the current components...
Processing epoch : 207 / 238
combining the current components...
Processing epoch : 208 / 238
combining the current components...
Processing epoch : 209 / 238
combining the current components...
Processing epoch : 210 / 238
combining the current components...
Processing epoch : 211 / 238
combining the current components...
Processing epoch : 212 / 238
combining the current components...
Processing epoch : 213 / 

In [ ]:
#my_load_data = np.load(op.join(save_folder, file[4:10]+"_EO.npz"), allow_pickle=True)
#print(my_load_data["left"].shape)
#print(my_load_data["right"].shape)

In [ ]:
"""
data_path = "Shared drives/Motor Imagery/resting state eeg & fmri/EEG_MPILMBB_LEMON/EEG_Preprocessed_BIDS_ID/EEG_Preprocessed/"
dirs = os.listdir(data_path)
print(len(dirs[740:]))
print(dirs[740:])
"""